![Title: Building Interactive Visualizations with Jupyter Notebook](presentation_title.png)

# What's Jupyter Notebook?
* A web application for scientific computation in the browser 
* Supports 40+ languages (kernals) including R and Matlab
* Formerly known as IPython Notebook
* Standard day-to-day tool for many scientists

# Problem Statement
Python has a wealth of  fantastic libraries for analyzing and plotting data, including
* matplotlib
* numpy
* pandas

However, the plots become static images when rendered in Jupyter or on the web.

**Wouldn't it be more interesting to have interactive plots?**

# Bokeh
For this demonstration, we'll use the [Bokeh](http://bokeh.pydata.org/en/latest/), pronounced "Bow (as in crossbow) Keh (as in kettle). We first need to tell  Bokeh to load its JavaScript component, BokehJS, using the ```output_notebook()``` function.

In [1]:
from bokeh.plotting import output_notebook, show

output_notebook()

Loading BokehJS ...

## Sample Data
A small set of data, ```autompg```, is available in the ```bokeh.sampledata``` package. ```autompg``` is in the ```pandas.dataframe``` format which you can think of as a database table or spreadsheet.

In [2]:
from bokeh.sampledata.autompg import autompg as df
#Display contents using qgrid
import qgrid
# copies javascript dependencies to your /nbextensions folder
qgrid.nbinstall(overwrite=True) 

qgrid.show_grid(df)

## Charts
Bokeh provides high level statistical plots such as bar charts, horizon plots and time series using the ```bokeh.charts``` interface. 

The interface is geared to be extremely simple to use in conjunction with the [Pandas](http://pandas.pydata.org/) data structures library (```Series``` or ```DataFrame```).

In the first plot below, we use the ```Bar``` chart to show the average miles per gallon for a given cylinder type. Note that we specify the ```mean``` aggregation type (default is sum).

In [3]:
from bokeh.charts import Bar

# group by cyl, apply aggregate average function to mpg column
barchart = Bar(df, label='cyl', values="mpg", agg='mean', title="Average MPG By Cycle")

show(barchart)

In [4]:
from bokeh.charts import Scatter

scatter = Scatter(df, x='mpg', y='hp', color='cyl', 
                  title="MPG vs Horsepower", xlabel="Miles Per Gallon",
                  ylabel="Horsepower")

show(scatter)

## Linking
Bokeh plots can be displayed in rows with their interactions linked.

In [5]:
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,wheel_zoom,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300, title="|x|")
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300, x_range=left.x_range, title="x^2")
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

## Widget
IPython widgets provide user interface components such as sliders and buttons styled to make your plots interactive.

In [6]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import push_notebook, show

x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5))
r = p.line(x, y, color="#2222aa", line_width=3)
widget_plot_target = show(p, notebook_handle=True)

def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook(handle=widget_plot_target)

In [7]:
from ipywidgets import interact

interact(update, f=["sin", "cos", "tan"], w=(0,100), A=(1,5), phi=(0, 20, 0.1))

<function __main__.update>

In [ ]:
## Server
The Bokeh Server can be used to support long running processes such as animations. Bokeh server is started on the commandline with 
```
bokeh serve
```
Bokeh server can also be started within the jupyter notebook but may affect system performance if a notebook using animations is restarted many times.

In [8]:
from numpy import pi, cos, sin, linspace, roll

from bokeh.client import push_session
from bokeh.io import curdoc, push_notebook
from bokeh.plotting import figure

M = 5
N = M*10 + 1
r_base = 8
theta = linspace(0, 2*pi, N)
r_x = linspace(0, 6*pi, N-1)
rmin = r_base - cos(r_x) - 1
rmax = r_base + sin(r_x) + 1

colors = ["FFFFCC", "#C7E9B4", "#7FCDBB", "#41B6C4", "#2C7FB8", "#253494", "#2C7FB8", "#41B6C4", "#7FCDBB", "#C7E9B4"] * M

# figure() function auto-adds the figure to curdoc()
p = figure(x_range=(-11, 11), y_range=(-11, 11))
r = p.annular_wedge(0, 0, rmin, rmax, theta[:-1], theta[1:],
                    fill_color=colors, line_color="white")

ds = r.data_source

roll_target = show(p, notebook_handle=True)

In [ ]:
def update_rolling_plot():
    rmin = roll(ds.data["inner_radius"], 1)
    rmax = roll(ds.data["outer_radius"], -1)
    ds.data.update(inner_radius=rmin, outer_radius=rmax)
    push_notebook(handle=roll_target)

# open a session to keep our local document in sync with server
session = push_session(curdoc())

ds = r.data_source

curdoc().add_periodic_callback(update_rolling_plot, 30)

session.loop_until_closed() # run forever

## Custom Tools
If you're building apps around bokeh, you might want to customize some of the tools. Here's an example of how to do that.

In [ ]:
from bokeh.core.properties import Instance
from bokeh.io import show
from bokeh.models import ColumnDataSource, Tool, CustomJS
from bokeh.plotting import figure

JS_CODE = """
import * as p from "core/properties"
import {GestureTool, GestureToolView} from "models/tools/gestures/gesture_tool"

export class DrawToolView extends GestureToolView

  # this is executed when the pan/drag event starts
  _pan_start: (e) ->
    @model.source.data = {x: [], y: []}

  # this is executed on subsequent mouse/touch moves
  _pan: (e) ->
    frame = @plot_model.frame
    canvas = @plot_view.canvas

    vx = canvas.sx_to_vx(e.bokeh.sx)
    vy = canvas.sy_to_vy(e.bokeh.sy)
    if not frame.contains(vx, vy)
      return null

    x = frame.x_mappers['default'].map_from_target(vx)
    y = frame.y_mappers['default'].map_from_target(vy)

    @model.source.data.x.push(x)
    @model.source.data.y.push(y)
    @model.source.trigger('change')

  # this is executed then the pan/drag ends
  _pan_end: (e) -> return null

export class DrawTool extends GestureTool
  default_view: DrawToolView
  type: "DrawTool"

  tool_name: "Drag Span"
  icon: "bk-tool-icon-lasso-select"
  event_type: "pan"
  default_order: 12

  @define { source: [ p.Instance ] }
"""

class DrawTool(Tool):
    __implementation__ = JS_CODE
    source = Instance(ColumnDataSource)

source = ColumnDataSource(data=dict(x=[], y=[]))

# plot = figure(x_range=(0,10), y_range=(0,10), tools=[DrawTool(source=source)])
plot = Scatter(df, x='mpg', y='hp', color='cyl',
                  title="Auto MPG", xlabel="Miles Per Gallon",
                  ylabel="Horsepower", tools=[DrawTool(source=source), "save"])
# plot.title.text ="Drag to draw on the plot"
plot.line('x', 'y', source=source)

show(plot)

# Plotly
[Plotly](https://plot.ly/)  is an enterprise application providing advanced graphing and data analysis within the browser. The python library and associated javascript is open source. By default the service uses the ```plot.ly``` account and servers  to host data but you can also run without a server or with a paid enterprise service on your corporate network.